# Science Case #1:

Find all <source-type> within a given area of the sky with emission from
<molecule(s)> detected.


The science motivation might be to find all young stellar objects (YSOs) in Taurus with ALMA
detections of CO J=2-1. The return from this query would include images of the moment maps of
CO emission, peak intensity, resolution, noise, correlator setting, and other relevant information
from any Taurus YSOs (as identified by science keywords, abstract text, SIMBAD coordinate
matches) in the publicly accessible ALMA Science Archive. The user could identify which data
are appropriate for their study an pull the u,v data or image cubes of interest from the archive.

Optionally, the user could then create a query to retrieve the full set of science products for all
sources, or the sources of interest. This would allow exploration of continuum and other lines
detected in the same observations. The quantitative data are returned as tables in the user’s
Python environment (for example Jupyter notebook) which can then be manipulated (see Section
4), for example to produce a plot of continuum flux versus CO integrated intensity or CO
intensity versus [13CO/12CO] intensity ratio.



In [1]:
from astroquery.admit import ADMIT, ADMIT_FORM_KEYS
from astroquery.alma import Alma, tapsql
from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd
# display the whole table inthe notebook
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',25)
cols_to_exclude = ['id','project_abstract','obs_title','fcoverage','science_keyword']


In [2]:
a = ADMIT()
a.check()
a.key_description.show_in_notebook()

Found  /home/mpound/src/astroumd/study7/query/admit.db
Checking db.... 0
71 71 71
header    : 103 entries
alma      : 103 entries
win       : 103 entries
lines     : 26 entries
sources   : 268 entries


idx,Keyword,Description,Database Table
0,alma_id,ALMA ID,Window
1,win_id,Spectral window ID,Window
2,spw,Spectral window name,Window
3,nlines,Number of lines,Window
4,nsources,Number of sources,Window
5,nchan,Number of channels,Window
6,win_peak,Window peak flux,Window
7,win_rms,Window RMS noise,Window
8,win_snr,Window Signal to noise ratio (peak/rms),Window
9,bmaj,Beam major axis,Window


In [3]:
#a.keys

In [12]:
# correct result should have one row
result = a.query(formula="CO", transition="2-1", project_abstract="*Black Hole*")
len(result)

kwargs  {'formula': 'CO', 'transition': '2-1', 'project_abstract': '*Black Hole*'}
formula
transition
project_abstract
select * from alma inner join win on (win.a_id = alma.id)  inner join lines on (lines.w_id = win.id )  WHERE lines.formula='CO' AND lines.transition='2-1' AND alma.project_abstract LIKE '%Black Hole%'


1

In [13]:
result

,id,obs_id,target_name,s_ra,s_dec,frequency,t_min,project_abstract,obs_title,science_keyword,scientific_category,proposal_authors,id,a_id,spw,freqc,freqw,vlsr,nlines,nsources,nchan,peak_w,rms_w,bmaj,bmin,bpa,fcoverage,id,w_id,formula,transition,restfreq,vmin,vmax,mom0flux,mom1peak,mom2peak
0,13,uid://A001/X1288/Xbae,NGC3593,168.654583,12.817675,230.055098,58384.714766,We propose to use ALM...,Weighing Black Hole M...,"Surveys of galaxies, ...",Galaxy evolution,"Greene, Jenny; Seth, ...",13,13,spw16,230.055109,1.873035,628.0,1,30,1918,0.068626,0.001117,0.000093,0.000081,35.348176,0.113139,7,13,CO,2-1,230.538,-136.849,138.029,11534.2,639.522,70.2828


In [11]:
c = SkyCoord('11h15m30s', '+12d45m00s', frame='icrs')
size = 1.5*u.degree
result = a.query(formula="CO", transition="2-1", project_abstract="*Black Hole*", region = [c,size])
len(result)

kwargs  {'formula': 'CO', 'transition': '2-1', 'project_abstract': '*Black Hole*', 'region': [<SkyCoord (ICRS): (ra, dec) in deg
    (168.875, 12.75)>, <Quantity 1.5 deg>]}
formula
transition
project_abstract
region
select * from alma inner join win on (win.a_id = alma.id)  inner join sources on (sources.w_id = win.id)   inner join lines on (lines.w_id = win.id )  WHERE lines.formula='CO' AND lines.transition='2-1' AND alma.project_abstract LIKE '%Black Hole%' AND  sources.ra > 167.37499999999997 AND sources.ra < 170.37499999999997 AND sources.dec > 11.25 AND sources.dec < 14.25  AND sources.l_id > 0 


30

In [ ]:
#columns to include 
ci = [x for x in result.columns if x not in cols_to_exclude]
result[ci]

In [ ]:
result=a.query(source_name_alma="NGC3049",mom0flux=">0")
len(result) 

In [ ]:
result

In [ ]:
result=a.query(source_name_alma='NGC3593',formula="CO")
len(result)# should be 1

In [ ]:
result=a.query(source_name_alma='NGC3593',source_snr=">-2")
len(result)

In [ ]:
result.head()

In [ ]:
result

In [ ]:
result=a.query(source_name_alma='NGC3504',source_snr=">3")
len(result)

In [ ]:
a._out_colna

In [ ]:

sql = "select * from alma inner join win on (win.a_id = alma.id)\
inner join sources on (sources.w_id = win.id)\
WHERE alma.target_name='NGC3504'\
AND sources.snr_s>=3.0 AND sources.l_id = 0 AND alra < 165.797 and sources.ra> 165.786"
result=pd.DataFrame(a.sql(sql),columns = a._out_colnames)
len(result)

In [ ]:
result.head()

In [ ]:
result=a.query(source_name_alma='NGC3504',source_snr=">3",formula="CO")
len(result)

In [ ]:
result=a.query(source_name_alma='NGC5253',source_peak=">0")
len(result) 

In [ ]:
result

### Example of how to exclude columns listed in the first cell

In [ ]:
#columns to include 
ci = [x for x in result.columns if x not in cols_to_exclude]
result[ci]

In [ ]:
result=a.query(source_name_alma='NGC3504',formula="CS")
len(result)

In [ ]:
result[ci]

In [ ]:
result1 = a.query(source_name_alma="NGC3504")
#result2 = a.query(source_name_resolver='ngc3504')
sql = "select * from alma where alma.target_name='NGC3504'"
result2 = a.sql(sql)
result2=pd.DataFrame(result2,columns=a._colnames['alma'])
result3 = a.query(source_name_alma="NGC3504",mom0flux=">-1")

In [ ]:
print(len(result1),len(result2),len(result3))

In [ ]:
result1


In [ ]:
result=a.query(scientific_category="galaxy evolution")  # case sensitive! returns zero results
len(result)

In [ ]:
result=a.query(win_snr=">1",source_name_alma="NGC3049") 
len(result)

In [ ]:
result

-------------------
tinker code below

In [ ]:
_gen_numeric_sql = _gen_str_sql = _gen_pos_sql = _gen_spec_res_sql = _gen_band_list_sql = _gen_datetime_sql = _gen_pol_sql = None

In [ ]:
ADMIT_FORM_KEYS["Lines"].values()

In [ ]:
ADMIT_FORM_KEYS.keys()

In [ ]:
if False:
    result1 = Alma.query_tap("select * from ivoa.obscore WHERE target_name = 'NGC3504'")
    result2 = Alma.query_tap("select * from ivoa.obscore WHERE target_name = 'ngc3504'")
    coord = SkyCoord.from_name('ngc3504')
    print(coord)
    sqlcoord = tapsql._gen_pos_sql('SkyCoord.from_name','ngc3504')
    print(sqlcoord)
    sql = "select * from ivoa.obscore WHERE source_name_resolver = '" +sqlcoord+"'"
    print(sql)
    if False:
        result3 = Alma.query_tap(sql) # fails    

In [ ]:
c = SkyCoord('00h42m30s', '+41d12m00s', frame='icrs')

In [ ]:
c

In [ ]:
def parse_region(region):
    ra_dec = region[0]
    size = region[1]
    if not isinstance(ra_dec,SkyCoord):
        raise ValueError("ra_dec region value must be astropy SkyCoord")
    if not isinstance(size,u.Quantity):
        raise ValueError("region values must be astropy Quantities")
    ra_min = c.ra - size.to("degree")
    ra_max = c.ra + size.to("degree")
    dec_min = c.dec - size.to("degree")
    dec_max = c.dec + size.to("degree")
    sql = f" sources.ra > {ra_min.to('degree')} AND sources.ra < {ra_max.to('degree')} AND sources.dec > {dec_min.to('degree')} AND sources.dec < {dec_max.to('degree')} "
    return sql

In [ ]:
parse_region([c,1.5*u.degree])

In [ ]:
f"sources.ra > {ra_min.to('degree')} AND sources.ra < {ra_max.to('degree')} AND sources.dec > {dec_min.to('degree')} AND sources.dec < {dec_max.to('degree')}"